# 代码准备

首先准备baseline微调代码与预训练模型，**注意**，在下面所有环节涉及到`/cache`目录，都是为了节约工作目录空间。`/cache`目录会在每次重启后清空，若想要持久化存储，请首先对notebook扩容，具体方式可以参考[文档](https://support.huaweicloud.com/devtool-modelarts/modelarts_30_0040.html)，并将`/cache`路径改写为`/home/ma-user/work`。

下载代码与预训练模型大约需要几分钟时间。

In [1]:
# 下载代码的本地路径
chatglm2_6b_lora_path = "/home/ma-user/work/chatglm2_6b_lora"

In [2]:
# 下载微调代码

import moxing as mox

mox.file.copy_parallel("obs://dtse-models-guiyang1/competition/glm2/chatglm2_6b_lora/", chatglm2_6b_lora_path)

ModuleNotFoundError: No module named 'moxing'

In [3]:
# 下载预训练模型

import moxing as mox

mox.file.copy("obs://llm-mindspore-ei/wxp/chatglm2-6b/glm2_6b_ms.ckpt", "/home/ma-user/work/glm2_6b.ckpt")

安装依赖包

In [10]:
!pip install -r $chatglm2_6b_lora_path/mindformers/requirements.txt

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 83 kB 29.3 MB/s eta 0:00:01
  Attempting uninstall: mindpet
    Found existing installation: mindpet 1.0.1
    Uninstalling mindpet-1.0.1:
      Successfully uninstalled mindpet-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mindformers 0.8.0 requires mindpet==1.0.1, but you have mindpet 1.0.2 which is incompatible.
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.


进入工作目录，准备训练配置文件与环境

In [11]:
%cd $chatglm2_6b_lora_path/mindformers/scripts

/home/ma-user/work/chatglm2_6b_lora/mindformers/scripts


/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


拷贝配置文件、驱动脚本等文件至mf_standalone，方便修改为个性化版本

In [5]:
!rm -rf ./mf_standalone
!mkdir ./mf_standalone
!cp ../*.py ./mf_standalone
!cp -r ../configs ./mf_standalone
!cp -r ../mindformers ./mf_standalone

# 微调模型

进入工作目录

In [12]:
%cd mf_standalone

/home/ma-user/work/chatglm2_6b_lora/mindformers/scripts/mf_standalone


修改微调配置文件，我们使用的配置文件路径为：`$chatglm2_6b_lora_path/mindformers/scripts/mf_standalone/configs/glm2/run_glm2_6b_lora_910b.yaml`

主要需要修改的字段：

```yaml
output_dir: '/home/ma-user/work/output'
load_checkpoint: '/home/ma-user/work/glm2_6b.ckpt'  # 修改为下载的预训练模型路径
# 必须，配置为obs路径，微调结果会直接上传至配置的obs桶中
remote_save_url: "obs://ai-l/competition/glm2/output-test/"

train_dataset: &train_dataset
  data_loader:
    dataset_dir: "/home/ma-user/work/chatglm2_6b_lora/train.jsonl"  # 修改为实际训练数据路径
    origin_columns: ["input", "output"]  # 根据实际训练数据使用的字段进行修改
  tokenizer:
    type: ChatGLM2Tokenizer
    vocab_file: "/home/ma-user/work/chatglm2_6b_lora/tokenizer.model"  # 实际使用的tokenizer.model路径，我们在微调代码$chatglm2_6b_lora_path中预置了一个tokenizer.model，如需使用自定义tokenizer model也可配置相关参数

# 如需训练同时验证，需同步修改验证数据相关字段
eval_dataset: &eval_dataset
```

修改后，运行微调命令，出现` .........Training Over!.............`日志即代表微调成功。

In [13]:
!MS_ENABLE_FORMAT_MODE=1 GE_USE_STATIC_MEMORY=2 HCCL_CONNECT_TIMEOUT=6000 MS_ASCEND_CHECK_OVERFLOW_MODE="INFNAN_MODE" DEVICE_ID=0 RUN_STATUS="finetune" python run_mindformer.py --config="configs/glm2/run_glm2_6b_lora_910b.yaml" --use_parallel=False --run_mode="finetune"

/usr/local/Ascend/ascend-toolkit/7.0.RC1/python/site-packages/tbe/tvm/contrib/ccec.py:766: DeprecationWarning: invalid escape sequence \L
  if not dirpath.find("AppData\Local\Temp"):
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/classifier/transdata/transdata_classifier.py:222: DeprecationWarning: invalid escape sequence \B
  """
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/vector/transdata/common/graph/transdata_graph_info.py:140: DeprecationWarning: invalid escape sequence \c
  """
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, 

# 模型验证

可以看到，微调后的所有模型已经保存在配置的obs桶中，其中，最后一个模型同时也可以在配置中的`output_dir`本地路径看到。我们可以对微调后的模型进行测试。

In [17]:
%ll /home/ma-user/work/chatglm2_6b_lora/mindformers/scripts/mf_standalone/output/checkpoint/rank_0/ -h

total 13G
-r-------- 1 ma-user  13G Nov 17 10:42 glm2-6b-lora_rank_0-937_4.ckpt
-rw------- 1 ma-user 6.4M Nov 17 09:47 glm2-6b-lora_rank_0-graph.meta


In [20]:
finetuning_model_path = "/home/ma-user/work/ma-user-work/checkpoint/rank_0/glm2-6b-lora_rank_0-937_4.ckpt"

将tokenizer.model拷贝至默认文件夹下

In [18]:
!mkdir -p $chatglm2_6b_lora_path/mindformers/scripts/mf_standalone/checkpoint_download/glm2
!cp $chatglm2_6b_lora_path/tokenizer.model $chatglm2_6b_lora_path/mindformers/scripts/mf_standalone/checkpoint_download/glm2/tokenizer.model

修改eval配置文件，我们使用的配置文件路径为：`$chatglm2_6b_lora_path/mindformers/scripts/mf_standalone/configs/glm2/run_glm2_6b_lora_eval.yaml`

主要需要修改的字段：

```yaml
load_checkpoint: '/home/ma-user/work/glm2_6b.ckpt'
# 必须，配置为obs路径
remote_save_url: "obs://ai-l/competition/glm2/eval-output-test/"

eval_dataset: &eval_dataset
  data_loader:
    dataset_dir: "/home/ma-user/work/chatglm2_6b_lora/train.jsonl"  # 修改为实际数据路径
    origin_columns: ["input", "output"]  # 根据实际数据使用的字段进行修改
  tokenizer:
    type: ChatGLM2Tokenizer
    vocab_file: "/home/ma-user/work/chatglm2_6b_lora/tokenizer.model"  # 实际使用的tokenizer.model路径，我们在微调代码$chatglm2_6b_lora_path中预置了一个tokenizer.model，如需使用自定义tokenizer model也可配置相关参数
    
```
eval阶段较为耗时，建议准备的验证数据为10条左右，若验证数据少于batch_size，请将batch_size改小。

修改后，运行eval命令，**注意**修改ckpt为实际微调后checkpoint路径 `--load_checkpoint=$finetuning_model_path`

In [21]:
!python run_mindformer.py --config=configs/glm2/run_glm2_6b_lora_eval.yaml --use_parallel=False --run_mode="eval" --device_id=0 --load_checkpoint=$finetuning_model_path

/usr/local/Ascend/ascend-toolkit/7.0.RC1/python/site-packages/tbe/tvm/contrib/ccec.py:766: DeprecationWarning: invalid escape sequence \L
  if not dirpath.find("AppData\Local\Temp"):
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/classifier/transdata/transdata_classifier.py:222: DeprecationWarning: invalid escape sequence \B
  """
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/vector/transdata/common/graph/transdata_graph_info.py:140: DeprecationWarning: invalid escape sequence \c
  """
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, 

eval会打印验证数据的推理结果，如需对比微调前后效果，可以对预训练模型也进行验证，**此步骤不是必须的**

In [22]:
!python run_mindformer.py --config=configs/glm2/run_glm2_6b_lora_eval.yaml --use_parallel=False --run_mode="eval" --device_id=0 --load_checkpoint=/cache/glm2_6b.ckpt

/usr/local/Ascend/ascend-toolkit/7.0.RC1/python/site-packages/tbe/tvm/contrib/ccec.py:766: DeprecationWarning: invalid escape sequence \L
  if not dirpath.find("AppData\Local\Temp"):
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/classifier/transdata/transdata_classifier.py:222: DeprecationWarning: invalid escape sequence \B
  """
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/vector/transdata/common/graph/transdata_graph_info.py:140: DeprecationWarning: invalid escape sequence \c
  """
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, 

# 模型转换

模型效果满足要求，需转换为mindir格式进行推理，进入模型转换工具包路径，准备模型转换环境

In [23]:
%cd $chatglm2_6b_lora_path/mindspore_lite_devserver/

/home/ma-user/work/chatglm2_6b_lora/mindspore_lite_devserver


/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


安装模型转换依赖包

In [26]:
!pip install -r mindformers_requirements.txt

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 50.9 MB 52.6 MB/s eta 0:00:01
     |████████████████████████████████| 7.9 MB 59.1 MB/s eta 0:00:01
     |████████████████████████████████| 232 kB 72.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 53.8 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 47.9 MB/s eta 0:00:01
  Attempting uninstall: werkzeug
    Found existing installation: werkzeug 3.0.1
    Uninstalling werkzeug-3.0.1:
      Successfully uninstalled werkzeug-3.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mindinsight 2.2.0 requires protobuf<=3.20.1,>=3.13.0, but you have protobuf 3.20.2 which is incompatible.
flask 2.3.3 requires Werkzeug>=2.3.7, but you have werkzeug 2.2.2 which is incompatible.
You should consider upgrading via the '/hom

我们将微调后的模型转换到`/cache/mindir`目录下，注意修改微调模型路径为实际路径。

In [6]:
mindir_path = "/home/ma-user/work/mindir"

In [28]:
!python ckpt_to_mindir.py --ckpt_path=$finetuning_model_path --tokenize_model_path=$chatglm2_6b_lora_path/tokenizer.model --output_dir=$mindir_path --model_name=glm2 --train_mode=origin --size=6 --seq_length=2048

mv: cannot stat '/cache/ma-user-work/checkpoint/rank_0/glm2-6b-lora_rank_0-937_4.ckpt': No such file or directory
2023-11-17 11:15:18,718 - INFO - tokenizer save to path: /cache/mindir/tokenizer
2023-11-17 11:15:18,718 - INFO - convert finish, start export mindir, it takes about 5~10 minutes
2023-11-17 11:21:34,349 - INFO - mindir export success, output dir: /cache/mindir


看到`mindir export success`日志说明模型已经导出到`$mindir_path`路径下。如报错，可以查看`$mindir_path/logs/export_mindir.log`路径下的日志确定原因。

# AI应用开发与验证

最终的交付件为AI应用，AI应用规范可以查阅[帮助文档](https://support.huaweicloud.com/inference-modelarts/inference-modelarts-0008.html)，我们提供了样例模板，可以在模板上进行个性化修改。

In [2]:
# 下载AI应用模板路径
chatglm2_model_path = "/home/ma-user/work/chatglm2_model"

In [3]:
import moxing as mox

mox.file.copy_parallel("obs://dtse-models-guiyang1/competition/glm2/chatglm2_model/", chatglm2_model_path)

INFO:root:Using MoXing-v2.2.3.2c7f2141-2c7f2141
INFO:root:Using OBS-Python-SDK-3.20.9.1
INFO:root:Using OBS-C-SDK-2.23.1
INFO:root:An exception occurred in function copy_parallel_c: /home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/moxing/framework/file/csrc/obs/lib/aarch64/libobs_cxx.so: cannot open shared object file: No such file or directory
INFO:root:load shared library failed, retry with copy parallel python
INFO:root:Multiprocessing connection patch for bpo-17560 not applied, not an applicable Python version: 3.9.10 | packaged by conda-forge | (main, Feb  1 2022, 21:53:27) 
[GCC 9.4.0]
INFO:root:List OBS time cost: 0.81 seconds.
INFO:root:Copy parallel total time cost: 1.35 seconds.


由于我们上线的AI应用采用的是mindpet 1.0.1版本，因此我们需要将mindpet降为对应版本，**如后续需要再次运行微调步骤**，则需在运行前运行 `!pip install mindpet==1.0.2`

In [1]:
!pip install mindpet==1.0.1

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.


进入工作路径

In [4]:
%cd $chatglm2_model_path

/home/ma-user/work/chatglm2_model


/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


可以先在notebook中调试prompt_template与推理参数

In [7]:
import os
from mindformers import ChatGLM2Tokenizer
from mindformers.infer.infers import InferTask
from mindformers.infer.infer_config import InferConfig
from model_service.model_service import SingleNodeService

# 基础镜像环境变量配置不支持推理，移除相关配置
os.unsetenv('MS_ENABLE_GE')
os.unsetenv('MS_GE_TRAIN')


# 为了方便部署AI应用，我们在模型转换时将权重文件进行了切分，在实际推理时，需要先将权重合并

def cat_weight_file(mindir_dir: str):
    for filename in os.listdir(mindir_dir):
        if '_variables' in filename:
            weight_dir = os.path.join(mindir_dir, filename)
            if len(list(weight_dir)) > 1 and not os.path.exists(f"{weight_dir}/data_0"):
                os.system(f"cat {weight_dir}/data_0_* > {weight_dir}/data_0")

root = mindir_path
cat_weight_file(os.path.join(root, 'mindir'))
prefill_model_path = os.path.join(root, 'mindir', 'chatglm2_6b_seq2048_bs1_full_graph.mindir')
increment_model_path = os.path.join(root, 'mindir', 'chatglm2_6b_seq2048_bs1_inc_graph.mindir')
ge_config_path = os.path.join(root, 'config.ini')
tokenizer_path = os.path.join(chatglm2_model_path, 'checkpoint_download')

lite_config = InferConfig(
            prefill_model_path=prefill_model_path,
            increment_model_path=increment_model_path,
            model_type="mindir",
            model_name="glm2",
            ge_config_path=ge_config_path,
            device_id=0,
            infer_seq_length=2048,
        )

# 模型加载与热备会耗时10-15min
tokenizer = ChatGLM2Tokenizer.from_pretrained(tokenizer_path)
infer_model = InferTask.get_infer_task("text_generation", lite_config, tokenizer=tokenizer)
infer_model.infer('hello')

2023-11-17 15:30:52,060 - mindformers - INFO - config in the yaml file /home/ma-user/work/chatglm2_model/checkpoint_download/glm2_6b.yaml are used for tokenizer building.
2023-11-17 15:30:52,107 - mindformers - WARNING - Can't find the tokenizer_config.json in the file_dict. The content of file_dict is : {}
2023-11-17 15:30:52,108 - mindformers - INFO - build tokenizer class name is: ChatGLM2Tokenizer using args {'bos_token': '<sop>', 'eos_token': '<eop>', 'end_token': '</s>', 'mask_token': '[MASK]', 'gmask_token': '[gMASK]', 'pad_token': '<pad>', 'unk_token': '<unk>', 'vocab_file': '/home/ma-user/work/chatglm2_model/checkpoint_download/tokenizer.model'}.


/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/extract_image_patches_without_cbuf_schedule.py:317: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if _ is not 1:
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/extract_image_patches_without_cbuf_schedule.py:317: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if _ is not 1:
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/extract_image_patches_without_cbuf_schedule.py:317: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if _ is not 1:
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/extract_image_patches_without_cbuf_schedule.py:317: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if _ is not 1:
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/extract_image_patches_without_cbuf_schedule.py:317: SyntaxWarning: "

['helloHM\n\n0000000000000000000000000000 at should do. should use.;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n;;;;;;;;...................................................']

In [23]:
#使用 prompt_template 构建输入，此步并不是必须的，也可以使用Jinja2等其他方法渲染模板，Baseline只是提供一种样例。

prompt_template = "[Round {}]\n\n问：{}\n\n答："

def build_prompt(text, i=1):
    return prompt_template.format(i, text)

In [30]:
# 给定测试 prompt, 最好与实际任务贴切，方便调试效果

prompt = "下肢血管狭窄的影像学检查有什么？ \n选项：\nA.直接胆胰管造影 \nB.彩色多普勒超声 \nC.支气管镜检查 \nD.CT检查"

In [31]:
prompt = build_prompt(prompt)
input_ids = infer_model.preprocess(prompt, add_special_tokens=True)
input_token_length = len(input_ids[0])
if input_token_length > 2048:
    response = 'inputs are too long, please retry.'
else:
    output_ids, logits = infer_model.generate(input_ids, do_sample=True, top_k=1,
                                      repetition_penalty=1, eos_token_id=2, pad_token_id=0,
                                      is_sample_acceleration=False, streamer=None,
                                      top_p=1, temperature=1, max_length=2048, return_logits=True)
    outputs = infer_model.postprocess(output_ids)
    response = outputs[0]

# 查看解码后的输出结果
print(response)

【只需要给出对应选项，不需要给出原因】
[Round 1]

问：下肢血管狭窄的影像学检查有什么？ 
选项：
A.直接胆胰管造影 
B.彩色多普勒超声 
C.支气管镜检查 
D.CT检查

答： CT检查


我们要求AI应用返回值只包含选项如：A，选手可以通过构建特定形式的微调数据、调整prompt_template、添加后处理等方式规范输出，下面提供一种处理思路：

In [32]:
choices = ["A", "B", "C", "D"]
choice_tokens = [tokenizer.encode(choice, add_special_tokens=False)[0] for choice in choices]

In [33]:
extraction_prompt = f'【只需要给出对应选项，不需要给出原因】'
prompt_template = "[Round {}]\n\n问：{}\n\n答："

def build_prompt(text, i=1):
    return extraction_prompt + '\n' + prompt_template.format(i, text)

In [34]:
prompt = "下肢血管狭窄的影像学检查有什么？ \n选项：\nA.直接胆胰管造影 \nB.彩色多普勒超声 \nC.支气管镜检查 \nD.CT检查"
prompt = build_prompt(prompt)
print(prompt)

【只需要给出对应选项，不需要给出原因】
[Round 1]

问：下肢血管狭窄的影像学检查有什么？ 
选项：
A.直接胆胰管造影 
B.彩色多普勒超声 
C.支气管镜检查 
D.CT检查

答：


In [35]:
input_ids = infer_model.preprocess(prompt, add_special_tokens=True)
output_ids, logits = infer_model.generate(input_ids, do_sample=True, top_k=5,
                                          repetition_penalty=1, eos_token_id=2, pad_token_id=0,
                                          is_sample_acceleration=False, streamer=None,
                                          top_p=1, temperature=1, max_length=2048, return_logits=True)

读取指定位置logits在choice_tokens上的取值，较大的一个作为选项

In [36]:
logits = logits[0][:, choice_tokens]
preds = logits.argmax(axis=-1)
print(choices[preds[0]])

D


In [37]:
outputs = infer_model.postprocess(output_ids)
response = outputs[0]
print(response)

【只需要给出对应选项，不需要给出原因】
[Round 1]

问：下肢血管狭窄的影像学检查有什么？ 
选项：
A.直接胆胰管造影 
B.彩色多普勒超声 
C.支气管镜检查 
D.CT检查

答： CT


# 构建AI应用

调试好prompt_template与推理逻辑后，我们可以修改AI应用推理代码`$chatglm2_model_path/customize_service.py`，同时，也可以在Notebook中进行调试，当然调试并不是必须的。我们可以查看推理代码：

In [39]:
%pycat $chatglm2_model_path/customize_service.py

import os
import time
import uuid
import threading
import json
from mindformers import ChatGLM2Tokenizer
from mindformers.infer.infers import InferTask
from mindformers.infer.infer_config import InferConfig
from model_service.model_service import SingleNodeService


# 基础镜像环境变量配置不支持推理，移除相关配置
os.unsetenv('MS_ENABLE_GE')
os.unsetenv('MS_GE_TRAIN')


def cat_weight_file(mindir_dir: str):
    for filename in os.listdir(mindir_dir):
        if '_variables' in filename:
            weight_dir = os.path.join(mindir_dir, filename)
            if len(list(weight_dir)) > 1 and not os.path.exists(f"{weight_dir}/data_0"):
                os.system(f"cat {weight_dir}/data_0_* > {weight_dir}/data_0")



class LlmService(SingleNodeService):
    def __init__(self, model_name, model_path):
        # 获取程序当前运行路径，即model文件夹所在的路径
        root = os.path.dirname(os.path.abspath(__file__))
        cat_weight_file(os.path.join(root, 'mindir'))
        self.prefill_model_path = os.path.join(root, 'mindir', 'chatg

拷贝代码到工作目录

In [41]:
!cp -rf $chatglm2_model_path/* /home/ma-user/work/mindir/

In [1]:
%cd /cache/mindir/
%ll

/cache/mindir


/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


total 60
drwxr-x---  2 ma-user 4096 Nov 17 15:55 __pycache__/
-rw-r-----  1 ma-user  287 Nov 17 15:18 args.json
drwxr-x---  2 ma-user 4096 Nov 17 15:55 checkpoint_download/
-rw-r-----  1 ma-user  118 Nov 17 15:55 config.ini
-rw-r-----  1 ma-user 1613 Nov 17 15:55 config.json
-rw-r-----  1 ma-user 5783 Nov 17 15:55 customize_service.py
drwxr-x---  3 ma-user 4096 Nov 17 15:55 kernel_meta_11263499095870700066/
drwxr-x---  3 ma-user 4096 Nov 17 15:55 kernel_meta_14679507254913862661/
drwxr-x---  2 ma-user 4096 Nov 17 15:55 kernel_meta_temp_14205105289487976846/
drwxr-x---  2 ma-user 4096 Nov 17 15:55 kernel_meta_temp_3881696564625442621/
drwxr-x---  2 ma-user 4096 Nov 17 15:18 logs/
drwxr-x--- 14 ma-user 4096 Nov 17 15:55 mindformers/
drwxr-x---  4 ma-user 4096 Nov 17 15:18 mindir/
drwxr-x---  2 ma-user 4096 Nov 17 15:18 tokenizer/


初始化推理实例

In [3]:
from customize_service import LlmService

llm = LlmService("/cache/mindir/", "/cache/mindir/")

load and warm up start
2023-11-17 15:56:57,378 - mindformers - INFO - config in the yaml file /cache/mindir/checkpoint_download/glm2_6b.yaml are used for tokenizer building.
2023-11-17 15:56:57,423 - mindformers - WARNING - Can't find the tokenizer_config.json in the file_dict. The content of file_dict is : {}
2023-11-17 15:56:57,424 - mindformers - INFO - build tokenizer class name is: ChatGLM2Tokenizer using args {'bos_token': '<sop>', 'eos_token': '<eop>', 'end_token': '</s>', 'mask_token': '[MASK]', 'gmask_token': '[gMASK]', 'pad_token': '<pad>', 'unk_token': '<unk>', 'vocab_file': '/cache/mindir/checkpoint_download/tokenizer.model'}.


由于采用非阻塞的方式加载模型与热备，因此初始化实例后并不代表模型已经加载完成，可以通过health()函数判断，若热备完成，返回200，否则，返回404。热备同样需要10~15min。

In [13]:
print(llm.health())

('', 200)


等待模型热备完成，可以准备数据进行推理测试，由于我们的判分采用文件进行推理，在测试时我们可以模仿BytesIO输入

In [29]:
post_prompt = {
    "prompt": "请问下面描述的是什么类型的临床实验筛选标准？\n试验前3个月每日吸烟量多于5支者或使用其他含有等量尼古丁的烟草制品。\n选项：\nA.实验室检查 \nB.受体状态 \nC.特殊病人特征 \nD.吸烟状况",
    "choices": ["A", "B", "C", "D"]
}

In [31]:
import json
import io

post_data = {"file_name":{"input_txt": io.BytesIO(json.dumps(post_prompt).encode())}}

**预处理**

In [32]:
data = llm._preprocess(post_data)
data

{'prompt': '请问下面描述的是什么类型的临床实验筛选标准？\n试验前3个月每日吸烟量多于5支者或使用其他含有等量尼古丁的烟草制品。\n选项：\nA.实验室检查 \nB.受体状态 \nC.特殊病人特征 \nD.吸烟状况',
 'choices': ['A', 'B', 'C', 'D']}

**推理**

In [33]:
data = llm._inference(data)
data

{'result': {'id': '6fa06e75-322c-494f-a4b1-36fb245ec7bd',
  'created': 1700209120,
  'choice': 'D',
  'prompt_tokens': 66,
  'total_tokens': 95,
  'history': [('请问下面描述的是什么类型的临床实验筛选标准？\n试验前3个月每日吸烟量多于5支者或使用其他含有等量尼古丁的烟草制品。\n选项：\nA.实验室检查 \nB.受体状态 \nC.特殊病人特征 \nD.吸烟状况',
    '【只需要给出对应选项，不需要给出原因】\n[Round 1]\n\n问：请问下面描述的是什么类型的临床实验筛选标准？\n试验前3个月每日吸烟量多于5支者或使用其他含有等量尼古丁的烟草制品。\n选项：\nA.实验室检查 \nB.受体状态 \nC.特殊病人特征 \nD.吸烟状况\n\n答： D.吸烟状况')],
  'response': '【只需要给出对应选项，不需要给出原因】\n[Round 1]\n\n问：请问下面描述的是什么类型的临床实验筛选标准？\n试验前3个月每日吸烟量多于5支者或使用其他含有等量尼古丁的烟草制品。\n选项：\nA.实验室检查 \nB.受体状态 \nC.特殊病人特征 \nD.吸烟状况\n\n答： D.吸烟状况'}}

调试好应用推理代码，最后，我们删除一些构建AI应用时不需要的文件，将调试好的AI应用保存到obs中，推理时合并的文件不需要拷贝，之后就可以在ModelArts控制台中创建AI应用，具体方法可以参考指导手册。

In [35]:
!rm -rf __pycache__/ kernel_meta_* mindir/chatglm2_6b_seq2048_bs1_full_variables/data_0_* mindir/chatglm2_6b_seq2048_bs1_inc_variables/data_0_*

In [36]:
%ll

total 44
-rw-r-----  1 ma-user  287 Nov 17 15:18 args.json
drwxr-x---  2 ma-user 4096 Nov 17 15:55 checkpoint_download/
-rw-r-----  1 ma-user  118 Nov 17 15:55 config.ini
-rw-r-----  1 ma-user 1613 Nov 17 15:55 config.json
-rw-r-----  1 ma-user 5783 Nov 17 15:55 customize_service.py
drwxr-x---  2 ma-user 4096 Nov 17 15:18 logs/
drwxr-x--- 14 ma-user 4096 Nov 17 15:55 mindformers/
drwxr-x---  4 ma-user 4096 Nov 17 15:18 mindir/
drwxr-x---  3 ma-user 4096 Nov 17 15:56 output/
drwxr-x---  2 ma-user 4096 Nov 17 15:18 tokenizer/


将文件夹保存到obs桶中，记得修改路径为实际路径

In [4]:
import moxing as mox

mox.file.copy_parallel("/home/ma-user/work/mindir", "obs://zzaa/model_3")

ModuleNotFoundError: No module named 'moxing'

In [1]:
ls

 驱动器 C 中的卷是 Windows-SSD
 卷的序列号是 C828-3D4D

 C:\Users\consonnm\Desktop\华为云百模千态 的目录

2023/11/28  15:47    <DIR>          .
2023/11/28  14:23    <DIR>          ..
2023/11/28  15:47    <DIR>          .ipynb_checkpoints
2023/11/27  17:31               612 a.jsonl
2023/11/27  17:55             5,783 customize_service.py
2023/11/26  15:23           870,949 glm2-baseline-v1(1).ipynb
2023/11/25  21:27             7,455 train.jsonl
2023/11/26  16:04            12,916 配置文件.md
               5 个文件        897,715 字节
               3 个目录 51,451,584,512 可用字节


接下来，按照指导手册的介绍构建AI应用即可。